# Customer Churn Prediction Model


This project aims to leverage historical customer data and machine learning algorithms to build an accurate churn prediction model.  
By identifying customers likely to churn, companies can take targeted actions to address their concerns,   
offer tailored incentives, or improve their overall experience,   
ultimately reducing customer attrition and its associated costs.

## Data

This dataset contains customer information and attributes relevant to assessing customer churn, which refers to the likelihood of customers leaving a service or company. The dataset's primary objective is to provide insights into customer behavior and help determine whether a customer will churn or not. Each row in the dataset represents a customer with specific attributes, and the "Churn" column indicates whether the customer has churned (1) or not (0).

Dataset Attributes:
- **CustomerId**: The unique identifier for each customer.
- **Surname**: The last name of the customer.
- **CreditScore**: The credit score of the customer.
- **Geography**: The geographical location of the customer.
- **Gender**: The gender of the customer.
- **Age**: The age of the customer.
- **Tenure**: The number of years the customer has been with the company.
- **Balance**: The account balance of the customer.
- **NumOfProducts**: The number of products the customer has with the company.
- **IsActiveMember**: Whether the customer is an active member (1) or not (0).
- **Churn**: Whether the customer has churned (1) or not (0).

## Machine Learning Task:
This dataset is suitable for a supervised binary classification task, where machine learning models   
can be trained to predict customer churn based on the provided attributes.   
The models aim to classify customers as churned (1) or not churned (0).


## Exploratory Data Analysis (EDA)
EDA is a crucial aspect of data science that helps in understanding the underlying structure of the data. It involves analyzing and summarizing data visually and statistically to uncover patterns or relationships. The main goals of EDA include:

## Gaining a deeper understanding of the data
- Identifying data quality issues
- Developing initial insights and hypotheses
- Selecting features for modeling or further analysis

In [3]:
# import modules
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2024-06-20 13:40:35.482927: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading and Read the dataset

In [4]:
# read the csv
df = pd.read_csv('data/Customer_Churn.csv')

# number of lows and columns
rows = df.shape[0]
columns = df.shape[1]

# print the number of rows and columns
print('Number of rows: ', rows)
print('Number of columns: ', columns)

# view the first 5 rows
df.head()

Number of rows:  10000
Number of columns:  13


,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Churned
0,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
